In [ ]:
import numpy as np
import glob
import json
from tqdm.notebook import tqdm

from skimage import color



from utils import grainPreprocess, grainShow, grainMark, grainDraw, grainApprox, grainStats, grainMorphology,grainGenerate
from utils import SEMDataset

from crdp import rdp
from skimage import io, filters, morphology, util
from torch.utils.data import Dataset

from matplotlib import pyplot as plt
import os
from pathlib import Path
import json

from PIL import Image
from matplotlib import cm



## Углы

In [ ]:
images_path = '../../datasets/microstructures/original/o_bc_left'

dataset = SEMDataset(images_path, no_cache=None, max_images_num_per_class=100)


In [ ]:
%%time

step = 5

# images_path = '../../datasets/compressed/o_bc_left_4x_768_256x256_N360'
# json_save_name = 'o_bc_left_4x_768_256x256_N360'

# images_path = '../../datasets_generated/o_bc_left_4x_768_360_256_median_Ultra_Co6_2_N500'
# json_save_name = 'o_bc_left_4x_768_360_256_median_Ultra_Co6_2_N100'

images_path = '../../datasets_generated/diffusion/o_bc_left_4x_768_360_256_median_Ultra_Co6_2_N500'
json_save_name = 'o_bc_left_4x_768_360_256_median_Ultra_Co6_2_N500'

types_dict = {'Ultra_Co11': 'средние зерна',
              'Ultra_Co25': 'мелкие зерна',
              'Ultra_Co8': 'средне-мелкие зерна',
              'Ultra_Co6_2': 'крупные зерна',
              'Ultra_Co15': 'средне-мелкие зерна'}

# 256 x 256 
# 100, 250, 500, 1000, 10000


angles_approx_save(
                    images_path=images_path,
                    save_path=json_save_name,
                    types_dict=types_dict,
                    step=step,
                    max_images_num_per_class=360, 
                    no_cache = False,
                    workers = 20
                )

data = open(json_save_name+f'_step_{step}_degrees.json', encoding='utf-8')
data = json.load(data)

In [ ]:
step = 5

types_dict = {'Ultra_Co11': 'средние зерна',
            'Ultra_Co25': 'мелкие зерна',
            'Ultra_Co8': 'средне-мелкие зерна',
            'Ultra_Co6_2': 'крупные зерна',
            'Ultra_Co15': 'средне-мелкие зерна'}

# folders = ['diffusion/o_bc_left_4x_768_360_256_median_N500',]
folders = ['r3gan/o_bc_left_4x_768_360_median_rgb_256x256-gpus4-batch256_step23040_N10000',
           'r3gan/o_bc_left_4x_768_360_median_rgb_256x256-gpus8-batch512_step53248_N10000',
           'r3gan/o_bc_left_4x_768_360_median_rgb_256x256-gpus8-batch512_step94208_N10000']

sizes = [100,250, 500,1000,10000,]
# sizes = [100, 250, 500]

for folder in folders:
    for size in sizes:
        # images_path = '../../datasets_generated/o_bc_left_4x_768_360_256_median_Ultra_Co6_2_N500'
        images_path = f'../../datasets_generated/{folder}'
        json_save_name = f'{folder.split("/")[-1]}_N{size}'

        angles_approx_save(
                            images_path=images_path,
                            save_path=json_save_name,
                            types_dict=types_dict,
                            step=step,
                            max_images_num_per_class=size, 
                            no_cache = False,
                            workers = 20
                        )

# data = open(json_save_name+f'_step_{step}_degrees.json', encoding='utf-8')
# data = json.load(data)

In [ ]:
images_path = '../../datasets_generated/diffusion/o_bc_left_4x_768_360_768_median_N5000'

dataset = SEMDataset(images_path, no_cache=None, max_images_num_per_class=100)


In [ ]:

grainShow.angles_plot_base(data, plot_file_name=json_save_name, step=step, N=20, M=7, indices=[2,0,1], save=False)

## Сегменты контуров

In [ ]:
def contour_segment_approx_save(images_path, cached_base_dir,  save_path, types_dict, step,tol, max_images_num_per_class=None, no_cache=False, workers=None):
    pixel = 50 / 1000 
    json_data = []
    
    dataset = SEMDataset(images_path, cached_base_dir, no_cache=no_cache, max_images_num_per_class=max_images_num_per_class, workers=workers)
    shape=dataset.images_paths.shape
    
    pbar = tqdm(total=shape[0]*shape[1])
    types=[]
    names=[]

    for i in range(dataset.images_paths.shape[0]):
        all_lengths = []

        
        for j in range(dataset.images_paths.shape[1]):
            image, path = dataset.__getitem__(i,j)
            ang, cnts =grainMark.get_angles(image, tol=tol)
            name = path.split('/')[-2]
            type = types_dict[name]

            for cnt in cnts:
                for k, point in enumerate(cnt[:-1]):
                    y1, x1 = cnt[k - 1]
                    y2, x2 = cnt[k]
                    v = np.array((x1 - x2, y1 - y2)).reshape(1, 2)

                    dist = np.linalg.norm(v[0])
                    all_lengths.append(dist*pixel)

            pbar.update(1)

        names.append(name)
        types.append(type)


        x, y = grainStats.stats_preprocess(all_lengths, step)
        json_data.append((x,y))

    #     (x_gauss, y_gauss), mus, sigmas, amps = grainApprox.bimodal_gauss_approx(x, y)
    #     name = path.split('/')[-2]

    #     text = grainGenerate.angles_legend(dataset.images_paths.shape[1], types_dict[name], types_dict[name], step, mus, sigmas,amps, len(all_angles) )
        
    #     path='/'.join(path.split('/')[:-1])

    #     json_data.append({'path': path,
    #                       'name': name,
    #                       'type': types_dict[name],
    #                       'legend': text,
    #                       'density_curve_scatter': [x,y],
    #                       'gauss_approx_plot': [x_gauss, y_gauss],
    #                       'gauss_approx_data': {'mus': mus, 'sigmas':sigmas, 'amps':amps},
    #                       })

    with open(f'{save_path}_step_{step}_degrees.json', 'w', encoding='utf-8') as outfile:
        json.dump(json_data, outfile, cls=grainGenerate.NumpyEncoder, ensure_ascii=False)

    return json_data,types,names

In [ ]:
images_path = 'datasets/original/o_bc_left'
cached_base_dir = 'cv'

json_save_path = 'cv/o_bc_left_4x_768_tol_4'

types_dict = {'Ultra_Co11': 'средние зерна',
              'Ultra_Co25': 'мелкие зерна',
              'Ultra_Co8': 'средне-мелкие зерна',
              'Ultra_Co6_2': 'крупные зерна',
              'Ultra_Co15': 'средне-мелкие зерна'}

step=1
tol=5

data, types, names = contour_segment_approx_save(images_path,
                                        cached_base_dir,
                                        save_path=json_save_path, 
                                        step=step,
                                        tol = tol,
                                        types_dict=types_dict, 
                                        no_cache=False, 
                                        max_images_num_per_class=None)


In [ ]:
markers = ['v', 's', 'D', 'd', 'p', '*','P']
colors_1 = ['orange', 'red','blue','green', 'indigo', 'brown']

scatter_size = 20


plt.rcParams['font.size'] = '15'
plt.figure(figsize=(7,7))

indices = [2,0,3]

data_tmp = []
types_tmp = []
x_max=1

for i in indices:
    data_tmp.append(data[i])    

for i in range(len(data_tmp)):
    x = data_tmp[i][0]
    y = data_tmp[i][1]
    plt.scatter(x,np.log(y),  marker=markers[indices[i]],color=colors_1[indices[i]],s=scatter_size)



for i in range(len(data_tmp)):
    x = data_tmp[i][0]
    y = data_tmp[i][1]

    # x = x[10:-20]
    # y = y[10:-20]

    y = np.log(y)

    (x_pred1, y_pred1), k1, b1, angle1, score1 = grainApprox.lin_regr_approx(x.reshape(-1, 1), y.reshape(-1, 1))
    plt.plot(x_pred1, y_pred1)

    types_tmp.append(f'{types[indices[i]]} k={k1:.2f}')    

plt.legend(types_tmp)

plt.title(f'RDP_contour_reduction_tolerance_{tol}_step_{step}_мкм')
# plt.yscale('log', ) 
plt.ylabel('log( p(x) )')
plt.xlabel('contours segments len, мкм')
plt.savefig(f'pixels_distr_tol_{tol}.jpg',bbox_inches='tight')
plt.show()

In [ ]:
x

In [ ]:
images_path = 'datasets/original/o_bc_left'
cached_base_dir = 'cv'

max_images_num_per_class=None
no_cache=False
workers=None

dataset = SEMDataset(images_path,
                    cached_base_dir, 
                    no_cache=no_cache, 
                    max_images_num_per_class=max_images_num_per_class, 
                    workers=workers)

i=3
j=3
tol=3

image, path = dataset.__getitem__(i,j)
ang, cnts =grainMark.get_angles(image, tol=tol)


image_tmp = Image.fromarray(image).convert('RGB')
image_tmp = grainDraw.draw_contours(image_tmp, cnts)

plt.figure(figsize=(10,10))
plt.title(f'RDP_contour_reduction_tolerance_{tol}',fontsize=20)
plt.imshow(image_tmp)
plt.xlim(500,800)
plt.ylim(500,800)
plt.savefig(f'RDP_contour_reduction_tolerance_{tol}.jpg',bbox_inches='tight')